## PROJECT GOAL:
Outbrain is a web advertising platform that displays advertisement boxes of links to pages within websites. It displays links to the sites' pages in addition to sponsored content, generating revenue from the latter.
In our project we are predicting whether the particular ad_id will be clicked or not.

This python files predict ad using random forest classifier algorithm and measure Accuracy and AUC.

## Creating a Classification Model using Decision Tree Classifier Algorithm

In this file, we will implement a classification model using *decision tree Classifier* that uses features of a Event,Document and Train dataframe to **predict the whether the ad is clicked or not** 

You should follow the steps below to build, train and test the model from the source data:

1. Build a schema of a source data for its Data Frame
2. Load the Source Data to the schema
3. Prepare the data with the features (input columns, output column as label)
4. Split the data using data.randomSplit(): Training and Testing
5. Transform the columns to a vector using VectorAssembler
6. set features and label from the vector
7. Build a ***Decision Tree Classifier Algorithm*** Model with the label and features
8. Train the model
9. Prepare the testing Data Frame with features and label from the vector; Rename label to trueLabel
10. Predict and test the testing Data Frame using the model trained at the step 8
11. Compare the predicted result and trueLabel


### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [3]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier



#####The below command will start spark session when we run our file in oracle BDE. In Databricks keep this cell as False by default. But when you run file in Oracle BDE make it True.

In [5]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

### Load Source Data
The data for this project is provided as a CSV file containing details of advertisement. The data includes specific characteristics (or *features*) for each ad, as well as a *label* column indicating whether the ad was clicked or not.

You will load this data into a DataFrame and display it.

In [7]:
eventSchema = StructType([
  StructField("display_id", IntegerType(), False),
  StructField("uuid", StringType(), False),
  StructField("document_id", IntegerType(), False),
  StructField("timestamp", IntegerType(), False),
  StructField("platform", IntegerType(), False),
  StructField("geo_location", StringType(), False)
])

### Prepare the Data
Most modeling begins with exhaustive exploration and preparation of the data. In this example, you will simply select a subset of columns to use as *features* as well as the **ArrDelay** column, which will be the *label* your model will predict.

#####Reading all necessary csv file

In [10]:
if IS_SPARK_SUBMIT_CLI:
    event = spark.read.csv('events.csv', inferSchema=True, header=True)
else:
    event = spark.sql("SELECT * FROM eventd_csv")

In [11]:
#create train schema
trainSchema = StructType([
  StructField("display_id", DoubleType(), False),
  StructField("DayOfWeek", DoubleType(), False),
  StructField("clicked", IntegerType(), False)
])

In [12]:
#Read traind.csv file
if IS_SPARK_SUBMIT_CLI:
    train = spark.read.csv('trains.csv', inferSchema=True, header=True)
else:
    train = spark.sql("SELECT * FROM traind_csv")

In [13]:
# create Promoted_Content Schema
promoted_contentSchema = StructType([
  StructField("ad_id", IntegerType(), False),
  StructField("document_id", IntegerType(), False),
  StructField("campaign", IntegerType(), False),
  StructField("advertiser_id", IntegerType(), False)
])

In [14]:
# Read Promoted_content.csv file
if IS_SPARK_SUBMIT_CLI:
    promoted_content = spark.read.csv('promoted_content.csv', inferSchema=True, header=True)
else:
    promoted_content = spark.sql("SELECT * FROM promoted_content_csv")

Below command will join event and train file using disply_id as primary key. It will also drop display_id from event table and ad_id from train table as it create ambiguity error. Then it will assign result to new dataframe data1.

In [16]:
data1=event.join(train,event.display_id==train.display_id).drop(event.display_id).drop(train.ad_id)

Following cell will join two dataframe data1 (which created from joining of event and train table) with Promoted content dataframe using document_id as primary key. Then it will assign result to new dataframe data2.

In [18]:
data2=data1.join(promoted_content,data1.document_id==promoted_content.document_id).drop(data1.document_id)

Following table select disply_id,document_id,platform,ad_id,camaign_id,advertiser_id which are features which help us to train our dataset and clicked column which is label column. Selected column will assign to new dataframe called data.

In [20]:
data =  data2.select(col("display_id").cast(DoubleType()), col("document_id").cast(DoubleType()), col("platform").cast(DoubleType()), col("ad_id").cast(DoubleType()), col("campaign_id").cast(DoubleType()),col("advertiser_id").cast(DoubleType()),col("clicked").alias("label"))

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [22]:
splits = data.randomSplit([0.7, 0.3])     
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

### Prepare the Training Data create Pipeline
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector,Normlize data using Minmax.

In [24]:
# Import RandomClassifier Algorithm 
from pyspark.ml.classification import RandomForestClassifier

# Convert following feature column in one vector for train data
assembler = VectorAssembler(inputCols = ["display_id", "document_id", "platform", "ad_id","campaign_id","advertiser_id"],outputCol="normfeatures")
#assembler = VectorAssembler(inputCols = ["clicked"],outputCol="label")

#Normlize feature data 
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")

#Convert Normlize feature data to vector
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")

#following Random forest algorithm train the classifiction model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features",impurity='gini',maxDepth=30,maxBins=128,seed=1234)
# Following command will create pipeline with different stages
pipeline = Pipeline(stages=[assembler,minMax,featVect,dt])

The pipeline itself is an estimator, and so it has a fit method that you can call to run the pipeline on a specified DataFrame. In this case, you will run the pipeline on the training data to train a model.

In [26]:
piplineModel = pipeline.fit(train)
print("Pipeline complete!")


Pipeline complete!

### Test the Recommender 
The model produced by the pipeline is a transformed that will apply to all stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, you will transform the test DataFrame using the pipeline to generate label prediction

In [28]:
# piplineModel with train data set applies test data set and generate predictions
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+----------------------------------------------------------------------------------------------------------+----------+---------+
features |prediction|trueLabel|
+----------------------------------------------------------------------------------------------------------+----------+---------+
[0.2332248052029389,0.499579062617473,0.0,0.2847998432955505,0.5789429127747834,0.38717002554640934] |1.0 |1 |
[0.2332248052029389,0.499579062617473,0.0,0.3167161323383067,0.5789429127747834,0.38717002554640934] |1.0 |1 |
[0.2332248052029389,0.499579062617473,0.0,0.3167283748734209,0.5789429127747834,0.38717002554640934] |1.0 |1 |
[0.2332248052029389,0.499579062617473,0.0,0.31673187274059644,0.5789429127747834,0.38717002554640934] |1.0 |1 |
[0.5975214149875676,0.7984074416583646,0.0,0.8050131432359119,0.020966786127493864,0.002554640931024695] |0.0 |0 |
[0.8891742670492508,0.9745943921707076,0.5,0.9464738875470682,0.006885458701357338,2.838489923360772E-4] |1.0 |0 |
[0.10054107738780933,0.27431324316173283,0.5,0.07722766043405034,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.10054107738780933,0.27431324316173283,0.5,0.07722940936763809,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.10054107738780933,0.27431324316173283,0.5,0.07723465616840132,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18764500882897958,0.27431324316173283,0.5,0.07722416256687485,0.14343764991393176,0.01107011070110701] |0.0 |0 |
[0.18764500882897958,0.27431324316173283,0.5,0.07723290723481357,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18764500882897958,0.27431324316173283,0.5,0.07723465616840132,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18764500882897958,0.27431324316173283,0.5,0.07813185909891443,0.001890679233569433,0.01107011070110701]|0.0 |0 |
[0.5240636517633175,0.27431324316173283,0.0,0.07722416256687485,0.14343764991393176,0.01107011070110701] |1.0 |1 |
[0.5240636517633175,0.27431324316173283,0.0,0.07722940936763809,0.05909078082230437,0.01107011070110701] |1.0 |1 |
[0.5584698691221391,0.27431324316173283,1.0,0.07722940936763809,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.5584698691221391,0.27431324316173283,1.0,0.07723115830122583,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.5584698691221391,0.27431324316173283,1.0,0.07723465616840132,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.5584698691221391,0.27431324316173283,1.0,0.07813185909891443,0.001890679233569433,0.01107011070110701] |0.0 |0 |
[0.6055105061740524,0.27431324316173283,1.0,0.07722766043405034,0.05909078082230437,0.01107011070110701] |1.0 |1 |
[0.6055105061740524,0.27431324316173283,1.0,0.07722940936763809,0.05909078082230437,0.01107011070110701] |1.0 |1 |
[0.6677292113817108,0.8861360806941965,0.5,0.8842066050225875,0.0027372520247199257,0.04598353675844451] |0.0 |0 |
[0.9035938354755494,0.27431324316173283,1.0,0.07722940936763809,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9035938354755494,0.27431324316173283,1.0,0.07723290723481357,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9035938354755494,0.27431324316173283,1.0,0.07813185909891443,0.001890679233569433,0.01107011070110701] |0.0 |0 |
[0.9450187131624963,0.27431324316173283,1.0,0.07723115830122583,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450187131624963,0.27431324316173283,1.0,0.07813185909891443,0.001890679233569433,0.01107011070110701] |0.0 |0 |
[0.07565834446169642,0.6482510730687264,1.0,0.6265064876691437,0.003019442955103423,2.838489923360772E-4] |0.0 |0 |
[0.3926130425503685,0.48205406008041407,0.0,0.266911750560096,0.13502836018850353,0.4175418677263696] |0.0 |0 |
[0.9410909822711003,0.48205406008041407,0.0,0.266911750560096,0.13502836018850353,0.4175418677263696] |0.0 |0 |
[0.6936543842584444,0.896892138547716,1.0,0.8925874947750609,0.7301408132742614,0.027817201248935566] |0.0 |0 |
[0.17860570094736777,0.6930255178101536,0.0,0.6719665184153962,0.0027372520247199257,0.04598353675844451] |0.0 |0 |
[0.24420806175328783,0.6930255178101536,

Following cell measure accuracy of Algorithm using MultiClassificationEvaluater and evaluate using predicted data.

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print("Accuracy for Decision Tree classifier is=",accuracy)

Accuracy for Decision Tree classifier is= 0.943342776203966

Following Code will give total Error in project

In [32]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0566572

#### Decision Tree Evaluator
Calculate AUC using Decision Tree Evaluator.

In [34]:
rf_evaluator =  MulticlassClassificationEvaluator(labelCol="trueLabel", predictionCol="prediction")
rf_auc = rf_evaluator.evaluate(prediction)
print("AUC for Decision Tree is= ", rf_auc)

AUC for Decision Tree is= 0.9410911194259339

#### Decision Tree Classifier Confusion matrix
Calculate confusion matrix and measure precision,Recall

In [36]:
# Only for Classification Logistic Regression not for Linear Regression

tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()


+---------+------------------+
 metric| value|
+---------+------------------+
 TP| 1571.0|
 FP| 213.0|
 TN| 11749.0|
 FN| 587.0|
Precision|0.8806053811659192|
 Recall|0.7279888785912882|
+---------+------------------+